In [19]:
import json
import sqlite3
import pandas as pd
import pickle

with open('../env_keys.json') as infile:
    env_variables = json.load(infile)

In [11]:
db_path = '../beer.db'

In [17]:
conn = sqlite3.connect(db_path)
query = "SELECT * FROM user_extract WHERE user_extract.username = 'tsharp93'"
df = pd.read_sql(query,conn)

In [18]:
df

,username,beer_name,beer_description,brewery,ABV,IBU,global_rating,user_rating,first_date,recent_date,total
0,tsharp93,Firestone Lager,Lager - Helles,Firestone Walker Brewing Company,4.5,17,3.58,3.50,06/23/19,06/23/19,1
1,tsharp93,Köstritzer Schwarzbier,Schwarzbier,Köstritzer Schwarzbierbrauerei,4.8,22,3.53,3.25,06/11/19,06/11/19,1
2,tsharp93,Midas Touch,Gruit / Ancient Herbed Ale,Dogfish Head Craft Brewery,9.0,12,3.78,3.25,06/01/19,06/01/19,1
3,tsharp93,Twisted Monkey,Belgian Blonde,Victory Brewing Company,5.8,15,3.46,4.00,06/01/19,06/01/19,1
4,tsharp93,Fresh Squeezed IPA,IPA - American,Deschutes Brewery,6.4,60,3.94,4.25,05/21/19,05/21/19,1
5,tsharp93,Head High,IPA - American,Kane Brewing Company,6.6,80,3.97,3.50,05/18/19,05/18/19,1
6,tsharp93,Sig Sour 6.5,Sour - Other,Brew Republic Bierwerks,6.5,45,3.58,4.50,05/12/19,05/12/19,1
7,tsharp93,The Drongo,Sour - Other,Clown Shoes,6.5,None,3.54,4.25,05/11/19,05/11/19,1
8,tsharp93,Forbidden Planet,IPA - Imperial / Double,Raduga,8.4,115,3.67,4.50,05/09/19,05/09/19,1
9,tsharp93,They Both Melt,Lager - Munich Dunkel,Bluejacket,5.6,None,3.57,3.25,05/09/19,05/09/19,1


## Prep Data

In [28]:
x = df[['beer_description', 'brewery', 'ABV', 'IBU', 'global_rating', 'total']]
y = df['user_rating']

#### Descriptions

In [29]:
description_dummies = pd.get_dummies(x['beer_description'])

description_dummies.sum().sort_values(ascending=False).head(15)

IPA - American                       4
Sour - Other                         2
Stout - Russian Imperial             2
Strong Ale - American                1
Lager - Munich Dunkel                1
Cream Ale                            1
Fruit Beer                           1
Gruit / Ancient Herbed Ale           1
IPA - Imperial / Double              1
IPA - Session / India Session Ale    1
Lager - Helles                       1
Pale Ale - Belgian                   1
Porter - American                    1
Porter - Imperial / Double           1
Schwarzbier                          1
dtype: int64

In [30]:
# consolidate dummies, write new col to x
description_names = description_dummies.sum()[description_dummies.sum() >= 100].index.tolist()
x['description_consolidated'] = [val if val in description_names else 'Other' for val in x['beer_description']]

# drop Other, replace variables with dummies
description_dummies = pd.get_dummies(x['description_consolidated']).drop('Other', axis=1)

C:\Users\tomsharp\AppData\Local\Continuum\miniconda3\envs\ml_guild\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


#### Breweries

In [31]:
# consolidate dummies, write new col to x
description_names = description_dummies.sum()[description_dummies.sum() >= 100].index.tolist()
x['description_consolidated'] = [val if val in description_names else 'Other' for val in x['beer_description']]


# drop Other, replace variables with dummies
description_dummies = pd.get_dummies(x['description_consolidated']).drop('Other', axis=1)

C:\Users\tomsharp\AppData\Local\Continuum\miniconda3\envs\ml_guild\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
df.head()

,username,beer_name,beer_description,brewery,ABV,IBU,global_rating,user_rating,first_date,recent_date,total
0,tsharp93,Firestone Lager,Lager - Helles,Firestone Walker Brewing Company,4.5,17,3.58,3.50,06/23/19,06/23/19,1
1,tsharp93,Köstritzer Schwarzbier,Schwarzbier,Köstritzer Schwarzbierbrauerei,4.8,22,3.53,3.25,06/11/19,06/11/19,1
2,tsharp93,Midas Touch,Gruit / Ancient Herbed Ale,Dogfish Head Craft Brewery,9.0,12,3.78,3.25,06/01/19,06/01/19,1
3,tsharp93,Twisted Monkey,Belgian Blonde,Victory Brewing Company,5.8,15,3.46,4.00,06/01/19,06/01/19,1
4,tsharp93,Fresh Squeezed IPA,IPA - American,Deschutes Brewery,6.4,60,3.94,4.25,05/21/19,05/21/19,1


### Read In Model

In [24]:
with open('../models/lasso.pkl', 'rb') as file:
    lasso = pickle.load(file)
lasso

LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=True,
        positive=False, precompute='auto', random_state=12, selection='cyclic',
        tol=0.0001, verbose=False)